In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score, confusion_matrix

class GraphTransformerLayer(nn.Module):
    def __init__(self, generatedHiddenDimensions, generatedNumHeads):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=generatedHiddenDimensions, num_heads=generatedNumHeads, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(generatedHiddenDimensions, generatedHiddenDimensions * 4),
            nn.ReLU(),
            nn.Linear(generatedHiddenDimensions * 4, generatedHiddenDimensions)
        )
        self.norm1 = nn.LayerNorm(generatedHiddenDimensions)
        self.norm2 = nn.LayerNorm(generatedHiddenDimensions)

    def forward(self, generatedNodeEmbeddings, generatedSpatialAttentionBias):
        generatedAttentionOutput, _ = self.attn(generatedNodeEmbeddings, generatedNodeEmbeddings, generatedNodeEmbeddings, attn_mask=generatedSpatialAttentionBias)
        generatedNodeEmbeddings = self.norm1(generatedNodeEmbeddings + generatedAttentionOutput)
        generatedOutputValue = self.norm2(generatedNodeEmbeddings + self.ffn(generatedNodeEmbeddings))
        return generatedOutputValue

class GraphormerEncoder(nn.Module):
    def __init__(self, generatedInputFeatureDimension, generatedHiddenDimension, generatedNumLayers, generatedNumHeads, generatedUnreachableToken=1000):
        super().__init__()
        self.input_projection = nn.Linear(generatedInputFeatureDimension, generatedHiddenDimension)
        self.transformer_layers = nn.ModuleList([
            GraphTransformerLayer(generatedHiddenDimension, generatedNumHeads) for _ in range(generatedNumLayers)
        ])
        self.unreachable_token = generatedUnreachableToken

    def forward(self, generatedInputFeatures, generatedDistanceMatrix):
        r = self.input_projection(generatedInputFeatures)
        generatedAttentionBias = -torch.abs(generatedDistanceMatrix).float()
        generatedAttentionBias[generatedDistanceMatrix == self.generatedUnreachableToken] = float("-inf")
        for generatedLayerValue in self.transformer_layers:
            a = generatedLayerValue(r, generatedAttentionBias)
        return a.mean(dim=1)

class GraphormerMultitaskModel(nn.Module):
    def __init__(self, encoder: GraphormerEncoder, generatedHiddenDimension: int):
        super().__init__()
        self.encoder = encoder
        self.binary_classifier = nn.Sequential(
            nn.Linear(generatedHiddenDimension, 64), nn.ReLU(), nn.Linear(64, 1)
        )
        self.regression_head = nn.Sequential(
            nn.Linear(generatedHiddenDimension, 64), nn.ReLU(), nn.Linear(64, 1)
        )

    def forward(self, generatedFeatures, generatedDistances):
        generatedGraphRepresentation = self.encoder(generatedFeatures, generatedDistances)
        generatedBinaryLogit = self.binary_classifier(generatedGraphRepresentation)
        generatedPredictedScores = self.regression_head(generatedGraphRepresentation)
        return generatedBinaryLogit, generatedPredictedScores

class GraphormerBinaryClassifier(nn.Module):
    def __init__(self, encoder: GraphormerEncoder, generatedHiddenDimension: int):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(generatedHiddenDimension, 64), nn.ReLU(), nn.Linear(64, 1)
        )

    def forward(self, generatedFeatures, generatedDistances):
        generatedGraphRepresentation = self.encoder(generatedFeatures, generatedDistances)
        return self.classifier(generatedGraphRepresentation)

class GraphormerRegressor(nn.Module):
    def __init__(self, encoder: GraphormerEncoder, generatedHiddenDimension: int):
        super().__init__()
        self.encoder = encoder
        self.regressor = nn.Sequential(
            nn.Linear(generatedHiddenDimension, 64), nn.ReLU(), nn.Linear(64, 1)
        )

    def forward(self, generatedFeatures, generatedDistances):
        generatedGraphRepresentation = self.encoder(generatedFeatures, generatedDistances)
        return self.regressor(generatedGraphRepresentation)

def generatedGraphormerPredictions(
    generatedTrueBinaryLabels, predictedBinaryScores,
    generatedTrueRegressionLabels, predictedRegressionScores
):
    generatedBinaryPredictions = (predictedBinaryScores > 0.5).astype(int)
    return {
        "F1 Score": f1_score(generatedTrueBinaryLabels, generatedBinaryPredictions),
        "AUROC": roc_auc_score(generatedTrueBinaryLabels, predictedBinaryScores),
        "Confusion Matrix": confusion_matrix(generatedTrueBinaryLabels, generatedBinaryPredictions).tolist(),
        "RMSE": mean_squared_error(generatedTrueRegressionLabels, predictedRegressionScores, squared=False),
        "R² Score": r2_score(generatedTrueRegressionLabels, predictedRegressionScores)
    }